In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2

In [2]:
model_four_corner = tf.keras.models.load_model('D:/Github/Spliddit/models/four_corner_model/full_model')
model_corner_refiner = tf.keras.models.load_model('D:/Github/Spliddit/models/corner_refiner_model/full_model')

# Preparing the test data

In [147]:
test_path = 'D:/Github/Spliddit/testset/'
files = os.listdir(test_path)
index = 6
test_img = cv2.imread(test_path + files[index], 1)
original_img = test_img.copy()
test_img = cv2.resize(test_img, (100, 100))
test_img = test_img.astype('float32') / 255
test_img = test_img.reshape((-1, 100, 100, 3))

In [148]:
test_img.shape

(1, 100, 100, 3)

# Four Corner Predict

In [149]:
four_corners = model_four_corner.predict(test_img).reshape((8))

1.) Upscaling

In [150]:
def upscale_result(four_corners, img):
    height, width = img.shape[:2]
    pairs = get_coord_pairs(four_corners)
    scaled = []
    for pair in pairs:
        scaled.append((int(pair[0] * width), int(pair[1] * height)))
    return scaled
    
def get_coord_pairs(coords):
    return [(coords[0], coords[1]), (coords[2], coords[3]), (coords[4], coords[5]), (coords[6], coords[7])]

In [151]:
four_corners = upscale_result(four_corners, original_img)

2.) Extract corners

In [152]:
def extract_corners(img, coords):
    height, width = img.shape[:2]
    tl, tr, br, bl = four_corners
    tl_tr_x_diff = abs(tl[0] - tr[0])
    tl_bl_y_diff = abs(tl[1] - bl[1])
    tr_br_y_diff = abs(tr[1] - br[1])
    br_bl_x_diff = abs(br[0] - bl[0])
    
    tl_x = int(tl[0] + 0.5*tl_tr_x_diff)
    tl_y = int(tl[1] + 0.5*tl_bl_y_diff)
    tr_y = int(tr[1] + 0.5*tr_br_y_diff)
    br_x = int(bl[0] + 0.5*br_bl_x_diff)
    
    crop_tl = img[0:tl_y, 0:tl_x]
    crop_tr = img[0:tr_y, tl_x:width]
    crop_br = img[tr_y:height, br_x:width]
    crop_bl = img[tl_y:height, 0:br_x]
    
    return crop_tl, crop_tr, crop_br, crop_bl

In [153]:
crop_tl, crop_tr, crop_br, crop_bl = extract_corners(original_img, four_corners)

In [154]:
tl_height, tl_width = crop_tl.shape[:2]
tr_height, tr_width = crop_tr.shape[:2]
br_height, br_width = crop_br.shape[:2]
bl_height, bl_width = crop_bl.shape[:2]

# Refine Corners

In [155]:
def upscale_result(corner, img):
    height, width = img.shape[:2]
    x_scaled = int(corner[0] * width)
    y_scaled = int(corner[1] * height)
    return (x_scaled, y_scaled)

In [156]:
tl = cv2.resize(crop_tl, (100,100)).reshape((-1, 100, 100, 3))
tr = cv2.resize(crop_tr, (100,100)).reshape((-1, 100, 100, 3))
br = cv2.resize(crop_br, (100,100)).reshape((-1, 100, 100, 3))
bl = cv2.resize(crop_bl, (100,100)).reshape((-1, 100, 100, 3))

tl = tl.astype('float32') / 255
tr = tr.astype('float32') / 255
br = br.astype('float32') / 255
bl = bl.astype('float32') / 255

In [157]:
tl_pred = model_corner_refiner.predict(tl)[0]
tr_pred = model_corner_refiner.predict(tr)[0]
br_pred = model_corner_refiner.predict(br)[0]
bl_pred = model_corner_refiner.predict(bl)[0]

In [158]:
tl_scaled = upscale_result(tl_pred, crop_tl)
tr_scaled = upscale_result(tr_pred, crop_tr)
br_scaled = upscale_result(br_pred, crop_br)
bl_scaled = upscale_result(bl_pred, crop_bl)

# Recursive Refinement

In [159]:
def crop_image(img, coords, retain_factor):
    
    left_diff = abs(0 - coords[0])
    right_diff = abs(img.shape[1] - coords[0])
    top_diff = abs(0 - coords[1])
    bottom_diff = abs(img.shape[0] - coords[1])
    
    left_crop = int(left_diff * (1-retain_factor))
    right_crop = int(right_diff * (1-retain_factor))
    top_crop = int(top_diff * (1-retain_factor))
    bottom_crop = int(bottom_diff * (1-retain_factor))
    
    cropped =  img[0 + top_crop : img.shape[0] - bottom_crop, 0 + left_crop : img.shape[1] - right_crop]
    return cropped, [top_crop, bottom_crop, left_crop, right_crop]

In [160]:
tl_diffs, tr_diffs, br_diffs, bl_diffs = [], [], [], []
tl_original, tr_original, br_original, bl_original = crop_tl.copy(), crop_tr.copy(), crop_br.copy(), crop_bl.copy()

debug_coord = []
debug_img = []

while True:
    if ((tl_original.shape[0] * 0.75 < 100) or (tl_original.shape[1] * 0.75 < 100) or
        (tr_original.shape[0] * 0.75 < 100) or (tr_original.shape[0] * 0.75 < 100) or
        (br_original.shape[0] * 0.75 < 100) or (br_original.shape[0] * 0.75 < 100) or
        (bl_original.shape[0] * 0.75 < 100) or (bl_original.shape[0] * 0.75 < 100)):
        break
    
    crop_tl, tl_diff = crop_image(tl_original, tl_scaled, 0.75)
    crop_tr, tr_diff = crop_image(tr_original, tr_scaled, 0.75)
    crop_br, br_diff = crop_image(br_original, br_scaled, 0.75)
    crop_bl, bl_diff = crop_image(bl_original, bl_scaled, 0.75)
    
    tl_original, tr_original, br_original, bl_original = crop_tl.copy(), crop_tr.copy(), crop_br.copy(), crop_bl.copy()
    
    tl_diffs.append(tl_diff)
    tr_diffs.append(tr_diff)
    br_diffs.append(br_diff)
    bl_diffs.append(bl_diff)
    
    crop_tl = cv2.resize(crop_tl, (100, 100)).reshape((-1, 100, 100, 3)).astype('float32') / 255
    crop_tr = cv2.resize(crop_tr, (100, 100)).reshape((-1, 100, 100, 3)).astype('float32') / 255
    crop_br = cv2.resize(crop_br, (100, 100)).reshape((-1, 100, 100, 3)).astype('float32') / 255
    crop_bl = cv2.resize(crop_bl, (100, 100)).reshape((-1, 100, 100, 3)).astype('float32') / 255
    
    tl_pred =  model_corner_refiner.predict(crop_tl)[0]
    tr_pred =  model_corner_refiner.predict(crop_tr)[0]
    br_pred =  model_corner_refiner.predict(crop_br)[0]
    bl_pred =  model_corner_refiner.predict(crop_bl)[0]
    
    tl_scaled = upscale_result(tl_pred, tl_original)
    tr_scaled = upscale_result(tr_pred, tr_original)
    br_scaled = upscale_result(br_pred, br_original)
    bl_scaled = upscale_result(bl_pred, bl_original)
    
    debug_coord.append(br_scaled)
    debug_img.append(br_original)
    

In [161]:
#i = 3

#temp = debug_img[i].copy()
#cv2.circle(temp, debug_coord[i], 5,(255,0,0), thickness=-1)
#temp = cv2.resize(temp, (600, 800))
#cv2.imshow("Outline", temp)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Leading the refined prediction back to the original image

In [162]:
top_add_tl = [x[0] for x in tl_diffs]
left_add_tl = [x[2] for x in tl_diffs]

tl_coord_final = (tl_scaled[0] + sum(left_add_tl), tl_scaled[1] + sum(top_add_tl))

In [163]:
top_add_tr = [x[0] for x in tr_diffs]
left_add_tr = [x[2] for x in tr_diffs]

tr_coord_final = (tr_scaled[0] + sum(left_add_tr) + tl_width, tr_scaled[1] + sum(top_add_tr))

In [164]:
top_add_br = [x[0] for x in br_diffs]
left_add_br = [x[2] for x in br_diffs]

br_coord_final = (br_scaled[0] + sum(left_add_br) + bl_width, br_scaled[1] + sum(top_add_br) + tr_height)

In [165]:
top_add_bl = [x[0] for x in bl_diffs]
left_add_bl = [x[2] for x in bl_diffs]

bl_coord_final = (bl_scaled[0] + sum(left_add_bl), bl_scaled[1] + sum(top_add_bl) + tl_height)

In [166]:
temp = original_img.copy()
cv2.circle(temp, tl_coord_final, 5,(255,0,0), thickness=-1)
cv2.circle(temp, tr_coord_final, 5,(255,0,0), thickness=-1)
cv2.circle(temp, br_coord_final, 5,(255,0,0), thickness=-1)
cv2.circle(temp, bl_coord_final, 5,(255,0,0), thickness=-1)
temp = cv2.resize(temp, (600, 800))
cv2.imshow("Outline", temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Bild ausschneiden

In [167]:
top_left_x = min([tl_coord_final[0],tr_coord_final[0],br_coord_final[0],bl_coord_final[0]])
top_left_y = min([tl_coord_final[1],tr_coord_final[1],br_coord_final[1],bl_coord_final[1]])
bot_right_x = max([tl_coord_final[0],tr_coord_final[0],br_coord_final[0],bl_coord_final[0]])
bot_right_y = max([tl_coord_final[1],tr_coord_final[1],br_coord_final[1],bl_coord_final[1]])

img_cropped = original_img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]

# Ergebnis

In [168]:
def scale_size(x, y, max_x, max_y):
    """Scales two coordinates x and y so that
    they keep their ratio until either max_x or
    max_y is reached."""
    if (x > y):
        x_increment = 1
        y_increment = y/x
    elif (x < y):
        x_increment = x/y
        y_increment = 1
    else:
        x_increment = 1
        y_increment = 1
    if((x <= max_x) and (y <= max_y)):
        while(True):
            if((x + x_increment >= max_x) or (y + y_increment >= max_x)):
                return x, y
            else:
                x = x + x_increment
                y = y + y_increment
    else:
        while(True):
            if((x <= max_x) and (y  <= max_y)):
                return x, y
            else:
                x = x - x_increment
                y = y - y_increment

In [169]:
y, x, z = original_img.shape
x_scaled, y_scaled = scale_size(x, y, 1500, 800)
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', (int(x_scaled), int(y_scaled)))
cv2.imshow('image', original_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [177]:
y, x, z = img_cropped.shape
x_scaled, y_scaled = scale_size(x, y, 1500, 800)
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', (int(x_scaled), int(y_scaled)))
cv2.imshow('image', img_cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()